# Enhanced Stochastic Model for PLL-TRNG

This notebook implements an enhanced stochastic model for analyzing PLL-based True Random Number Generators (PLL-TRNG). The model considers:
- PLL design parameters
- Platform-specific characteristics
- Excess phase noise modeling
- Entropy analysis
- Correlation studies

This script uses Monte Carlo integration to solve the high-dimensional integration problem detailed in Sect. 4 of the paper. Monte Carlo integration is a numerical method, which uses random sampling to estimate integrals and works well for high-dimensional problems. Although Monte Carlo integration only provides approximate solutions, accuracy improves with more sample points. Three different sample sizes are recommended: 
- **$10^4$ (Low precision)**: fast computation time, suitable for quick estimates, larger error margin  
- **$10^5$ (Medium precision)**: balanced speed and accuracy, good for most applications, moderate error reduction  
- **$10^6$ (High precision)**: lower computation, best for final results, smallest error margin

Note:
To achieve the highest experimental accuracy, all data in the paper were obtained using high-precision Monte Carlo simulations with a size=$10^6$. The experimental results have been pre-saved in the `data` folder. High-precision simulations can be time-consuming. For quick verification, a lower precision can be used, though this may introduce significant errors especially in correlation analysis.

In [ ]:
# Import required packages
%pip install numpy matplotlib seaborn gmpy2 prettytable scipy

import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns
from numpy import pi
from gmpy2 import invert
from prettytable import PrettyTable
from mpl_toolkits.axes_grid1.inset_locator import inset_axes

# Plot Parameters
rc = {
    'figure.figsize': (6, 4),
    'figure.constrained_layout.use': True,
    'savefig.format': 'pdf',
    'font.size': 16,
}
mpl.rcParams.update(rc)

## Setup

### PLL-TRNG Parameters

Three PLL configurations for different performance:
- **Config1 (Recommended)**
  - Optimal correlation-sampling tradeoff
  - *Used as primary configuration in the paper*
- **Config2 (Speed Optimized)**
  - Fastest speed
  - Lower entropy
- **Config3 (Resolution Optimized)**
  - Highest coherent sampling resolution
  - Stronger point correlations

In [ ]:
# Config1: typical
pll_config1 = {
    "M0" : 43,
    "N0" : 4,
    "C0" : 7,
    "M1" : 32,
    "N1" : 3,
    "C1" : 3
}
# Config2: best R
pll_config2 = {
    "M0" : 49,
    "N0" : 4,
    "C0" : 8,
    "M1" : 16,
    "N1" : 1,
    "C1" : 3
}
# Config3: best S
pll_config3 = {
    "M0" : 53,
    "N0" : 5,
    "C0" : 7,
    "M1" : 28,
    "N1" : 3,
    "C1" : 3
}

Default design parameters and platform parameters are the followings.

In [ ]:
# PLL-TRNG Design Parameters
pll_fin = 1e8 # 100MHz
pll_nr_output = 2 # number of outputs
pll_phase_latency = 0.5 / pll_nr_output # latency between different outputs

# PLL-TRNG Platform Parameter
pll_BW = 1e6   # 1MHz
pll_L0 = 1e-10 # -100dBc

### Utility Functions

Here are some general-purpose utility functions used in the script:

In [ ]:
def mod_invert(x, base):
    return int(invert(x, base))

def print_class_instance_vars(obj):
    instance_vars = vars(obj)
    table = PrettyTable(['Attribute', 'Value'])
    for attr, value in instance_vars.items():
        table.add_row([attr, value])
    print(table)

def display_class_members(instance):
    table = PrettyTable()
    members = instance.__dict__.keys()
    values = instance.__dict__.values()
    pretty_values = [
        "{:.4e}".format(value) if isinstance(value, float) else value
        for value in values
    ]
    table.field_names = members
    table.add_row(pretty_values)
    print(table)

## Core Classes

This section define the main classes used in the stochastic model.

### Parameter Class

Classes for the design parameters and platform parameters for PLL-TRNGs as defined in Sect. 2.4 in the paper.

In [ ]:
class DesignParameter:
    def __init__(self, fin, pll_config, nr_output, phase_latency):
        # input frequency
        self.fin = fin
        # basic clk0 clk1 parameters
        self.f0  = (pll_config['M0'] * self.fin) / (pll_config['C0'] * pll_config['N0'])
        self.f1  = (pll_config['M1'] * self.fin) / (pll_config['C1'] * pll_config['N1'])
        self.KM  = pll_config['M1'] * pll_config['N0'] * pll_config['C0']
        self.KD  = pll_config['M0'] * pll_config['N1'] * pll_config['C1']
        self.S   = self.KD # ui
        self.R   = self.fin / (pll_config['N0'] * pll_config['N1'] * pll_config['C0'] * pll_config['C1'])
        self.T0  = 1 / self.f0
        self.T1  = 1 / self.f1
        self.Tp  = self.KD / self.f0
        self.delta = 1 / self.S
        self.nr_output = nr_output
        self.latency = phase_latency # ui

class PlatformParameter:
    def __init__(self, pll_duty, pll_initial_phase, L0, BW):
        self.alpha = pll_duty           # duty cycle
        self.phi_0 = pll_initial_phase  # initial phase
        self.L0    = L0
        self.BW    = BW
        self.sigma_a = np.sqrt(self.L0 * self.BW / (4 * pi)) # ui
        self.sigma_lt = np.sqrt(self.L0 * self.BW / (2 * pi)) # ui

### Excess Phase Class

Class for the excess phase process for PLLs as defined in Sect. 3.3 in the paper.

In [ ]:
class ExcessPhaseProcess:
    def __init__(self, designParameter, platformParameter):
        self.dp = designParameter
        self.pp = platformParameter
        self.dimension  = self.dp.KD * self.dp.nr_output
    # scalar
    def get_excess_phase_acf(self, ti, tj):
        A = self.pp.L0 * pi * self.pp.BW
        B = 2 * pi * self.pp.BW
        return A * np.exp(-B * np.abs(ti - tj))
    def get_excess_phase_var(self, ti):
        return self.get_excess_phase_acf(ti, ti)
    def get_excess_phase_corr(self, ti, tj):
        return self.get_excess_phase_acf(ti, tj) / (np.sqrt(self.get_excess_phase_var(ti) * self.get_excess_phase_var(tj)))
    # vector
    def get_time_vec(self):
        return np.array([self.dp.T0 * i - self.dp.latency * self.dp.T1 * j 
                        for i in range(self.dp.KD) 
                        for j in range(self.dp.nr_output)])
    # matrix
    def get_excess_noise_acf_matrix(self):
        t = self.get_time_vec()
        n = self.dimension
        return np.array([[self.get_excess_phase_acf(t[i], t[j]) for i in range(n)] for j in range(n)])
    def get_excess_noise_corr_matrix(self):
        t = self.get_time_vec()
        n = self.dimension
        return np.array([[self.get_excess_phase_corr(t[i], t[j]) for i in range(n)] for j in range(n)])
    # random variable
    def get_random_variable_instance(self, size):
        mean = np.zeros(self.dimension)
        cov  = self.get_excess_noise_acf_matrix() / (2 * pi)**2
        return np.random.multivariate_normal(mean, cov, int(size))

### Entropy Model Class

Class for the enhanced stochastic model for multiple-output PLL-TRNGs as described in Sect. 4.1 in the paper.

In [ ]:
class EntropyModel(ExcessPhaseProcess):
    def __init__(self, designParameter, platformParameter, size):
        ExcessPhaseProcess.__init__(self,designParameter,platformParameter)
        self.size = int(size)
        self.samples = self.get_random_variable_instance(self.size)
        self.contributors_arg_i = self.get_contributors_arg_i()
        self.contributors_arg_j = self.get_contributors_arg_j()
        self.contributors_cnt   = self.contributors_arg_i.size
        self.pr_R = self.get_pr_R()
    # Samples
    def get_output_sample(self, k):
        indices = np.arange(k, self.dimension, self.dp.nr_output)
        return self.samples[:, indices]
    # Reconstraction of sampled bits
    def get_j(self, i):
        return (i * self.dp.KM) % self.dp.KD
    def get_i(self, j):
        KM_invert = mod_invert(self.dp.KM, self.dp.KD)
        return (j * KM_invert) % self.dp.KD
    def get_phi_j(self, j, k):
        return self.pp.phi_0 - k * self.dp.latency + j * self.dp.delta
    def get_phi_i(self, i, k):
        return self.get_phi_j(self.get_j(i), k)
    def check_phi_i(self, excess_phase, i, k):
        phi = excess_phase + self.get_phi_i(i, k)
        return (0 < phi < self.pp.alpha) or (1 < phi < (1+self.pp.alpha)) or (-1 < phi < (-1 + self.pp.alpha)) or (2 < phi < (2+self.pp.alpha))
    # Probability and Entropy
    def get_pr_Xi(self, *args):
        if len(args) == 1:
            i, k = args[0] // self.dp.nr_output, args[0] % self.dp.nr_output
        elif len(args == 2):
            i, k = args
        samples = self.get_output_sample(k)
        phi_vec = samples[:, i]
        check_vec = np.vectorize(lambda s: self.check_phi_i(phi_vec[s], i, k))(np.arange(self.size))
        return check_vec.sum() / self.size
    def get_pr_Xi_vec(self):
        return np.array([self.get_pr_Xi(m) for m in range(self.dimension)])
    def get_pr_Xm0Xm1(self, m0, m1):
        cnt = 0
        for s in range(self.size):
            phi0 = self.samples[s, m0]
            phi1 = self.samples[s, m1]
            i0, k0 = m0 // self.dp.nr_output, m0 % self.dp.nr_output
            i1, k1 = m1 // self.dp.nr_output, m1 % self.dp.nr_output
            if self.check_phi_i(phi0, i0, k0) and self.check_phi_i(phi1, i1, k1):
                cnt = cnt + 1
        return cnt / self.size
    def get_pr_R(self):
        cnt = 0
        for s in range(self.size):
            r = False
            for m in range(self.dimension):
                phi = self.samples[s, m]
                i = m // self.dp.nr_output
                k = m % self.dp.nr_output
                if self.check_phi_i(phi, i, k):
                    r = not r
            cnt = cnt + 1 if r else cnt
        return cnt / self.size
    def get_shannon_entropy(self):
        p = self.pr_R
        return -p * np.log2(p) - (1-p) * np.log2(1-p) if p != 0 else 0
    def get_min_entropy(self):
        p = self.pr_R
        return -np.log2(np.maximum(p, 1-p)) if p != 0 else 0
    def print_entropy(self):
        shannon_entropy = self.get_shannon_entropy()
        min_entropy = self.get_min_entropy()
        print(f"Bias: {self.pr_R-0.5}")
        table = PrettyTable()
        table.field_names = ["Entropy", "Value"]
        table.add_row(["shannon", shannon_entropy])
        table.add_row(["min", min_entropy])
        print(table)
    # Contributors
    def get_contributors_arg_m(self):
        pr_Xj_vec = self.get_pr_Xi_vec()
        condition = (pr_Xj_vec >= 0.0225) & (pr_Xj_vec <= 0.9775)
        idxs = np.where(condition)[0]
        return idxs
    def get_contributors_arg_i(self):
        pr_Xj_vec = self.get_pr_Xi_vec()
        condition = (pr_Xj_vec >= 0.0225) & (pr_Xj_vec <= 0.9775)
        idxs = np.where(condition)[0]
        args_i = np.array([(m // self.dp.nr_output, m % self.dp.nr_output) for m in idxs])
        return args_i
    def get_contributors_arg_j(self):
        args_j = np.array([(self.get_j(i), k) for i, k in self.contributors_arg_i])
        return args_j
    def print_contributors_args(self):
        arg_zipped = np.array(list(zip(self.contributors_arg_i, self.contributors_arg_j)))
        arg = np.array([(item[0][0], item[1][0], item[0][1]) for item in arg_zipped], dtype=int) # i, j, k
        arg_sort = arg[np.lexsort((arg[:, 1], arg[:, 2]))]

        table = PrettyTable()
        table.field_names = ["#"] + [f"{i+1}" for i in range(len(arg_sort))]
        i_row = ["i"] + [data[0] for data in arg_sort]
        j_row = ["j"] + [data[1] for data in arg_sort]
        k_row = ["k"] + [data[2] for data in arg_sort]
        table.add_row(i_row)
        table.add_row(j_row)
        table.add_row(k_row)
        print(table)
    # Correlation of contributors
    def get_bits_cov(self, m0, m1):
        return self.get_pr_Xm0Xm1(m0, m1) - self.get_pr_Xi(m0) * self.get_pr_Xi(m1)
    def get_bits_var(self, m):
        p = self.get_pr_Xi(m)
        return p*(1-p)
    def get_bits_corr(self, m0, m1):
        var0 = self.get_bits_var(m0)
        var1 = self.get_bits_var(m1)
        cov  = self.get_bits_cov(m0, m1)
        return cov / np.sqrt(var0*var1)
    def get_contributors_corr_matrix(self):
        args = self.get_contributors_arg_m()
        n    = args.size
        return np.array([[self.get_bits_corr(args[i], args[j]) for j in range(n)] for i in range(n)])
    def plt_contributors_corr_matrix(self):
        m = self.get_contributors_corr_matrix()
        plt.figure(figsize=(10, 10))
        sns.heatmap(m, square=True, annot=True, cmap='coolwarm', fmt=".2f")
        plt.show()
    def plt_lag_corr(self):
        arg_m = self.get_contributors_arg_m()
        dimension = arg_m.size
        corr_martix = self.get_contributors_corr_matrix()
        t = self.get_time_vec()
        data = np.array([(t[arg_m[i]]-t[arg_m[j]], corr_martix[i][j]) for i in range(dimension) for j in range(dimension)])
        data_sort = sorted(data, key=lambda x: x[0])

        fig, ax = plt.subplots()
        x1, y1 = zip(*data_sort)
        ax.plot(x1, y1, 'C0o:', label='bit')

        x2 = np.linspace(-1.1 * max(x1), 1.1 * max(x1),1000)
        y2 = np.exp(-2 * pi * self.pp.BW * np.abs(x2))
        ax.plot(x2, y2, 'C2--', label='excess phase')

        ax.legend()
        ax.set_xlabel('lag')
        ax.set_ylabel('correlation')
        plt.show()

## Analysis and Visualization

### Reproduction of the paper figures

- Analyze the impact of PLL bandwidth on the correlation of the sampling points and reproduce the Fig. 7 in the paper:

In [ ]:
def plt_correlation_BW():
    # Config3 w/ nr_output = 1
    def get_data(BW):
        designParameter = DesignParameter(
            pll_fin,
            pll_config3,
            1,
            0.5)
        platformParameter = PlatformParameter(
            (designParameter.KD - 1) / (2 * designParameter.KD), # alpha
            designParameter.delta / 2, # phi_0
            1e-10,
            BW
        )
        excessPhase = ExcessPhaseProcess(
            designParameter,
            platformParameter
        )
        m = EntropyModel(
            designParameter,
            platformParameter,
            size=1e6
        )
        arg_m = m.get_contributors_arg_m()
        dimension = arg_m.size
        corr_martix = m.get_contributors_corr_matrix()
        t = m.get_time_vec()
        data = np.array([(t[arg_m[i]]-t[arg_m[j]], corr_martix[i][j]) for i in range(dimension) for j       in range(dimension)])
        data_sort = sorted(data, key=lambda x: x[0])
        return data_sort

    # Uncomment the following lines to generate data
    # data_sort1 = get_data(0.5e6)
    # data_sort2 = get_data(1e6)
    # data_sort3 = get_data(5e6)
    # x3, y3 = zip(*data_sort3)
    # x2, y2 = zip(*data_sort2)
    # x1, y1 = zip(*data_sort1)
    # np.savetxt('./data/correlation_bandwidth/x1.txt', x1)
    # np.savetxt('./data/correlation_bandwidth/x2.txt', x2)
    # np.savetxt('./data/correlation_bandwidth/x3.txt', x3)
    # np.savetxt('./data/correlation_bandwidth/y1.txt', y1)
    # np.savetxt('./data/correlation_bandwidth/y2.txt', y2)
    # np.savetxt('./data/correlation_bandwidth/y3.txt', y3)

    # Load precomputed data
    x1 = np.loadtxt('./data/correlation_bandwidth/x1.txt')
    x2 = np.loadtxt('./data/correlation_bandwidth/x2.txt')
    x3 = np.loadtxt('./data/correlation_bandwidth/x3.txt')
    y1 = np.loadtxt('./data/correlation_bandwidth/y1.txt')
    y2 = np.loadtxt('./data/correlation_bandwidth/y2.txt')
    y3 = np.loadtxt('./data/correlation_bandwidth/y3.txt')

    fig, ax = plt.subplots(figsize=(12, 4))

    ax.plot(x3, y3, 'C2o:', label='BW=5.0MHz')
    ax.plot(x2, y2, 'C1s:', label='BW=1.0MHz')
    ax.plot(x1, y1, 'C0^:', label='BW=0.5MHz')

    ax.legend(loc='upper left')
    ax.set_xlabel('lag[s]')
    ax.set_ylabel('ACF')

    axins = inset_axes(ax, width="40%", height="60%", loc="upper right")
    axins.plot(x3, y3, 'C2o:', label='BW=5.0MHz')
    axins.plot(x2, y2, 'C1s:', label='BW=1.0MHz')
    axins.plot(x1, y1, 'C0^:', label='BW=0.5MHz')
    axins.set_xlim(0.2e-6, 1e-6)
    axins.set_ylim(-0.1, 0.2)
    
    from matplotlib.ticker import MultipleLocator
    axins.xaxis.set_major_locator(MultipleLocator(0.1))
    axins.yaxis.set_major_locator(MultipleLocator(0.2))
    axins.set_xticks([])
    axins.set_yticks([])

    from mpl_toolkits.axes_grid1.inset_locator import mark_inset
    mark_inset(ax, axins, loc1=2, loc2=4, fc="none", ec="0.5")

    plt.show()

plt_correlation_BW()

- Analyze the impact of jitter indensity on the correlation of the sampling points and reproduce the Fig. 8 in the paper:

In [ ]:
def plt_correlation_L0():
    # Config3 w/ nr_output = 1
    def get_data(L0):
        designParameter = DesignParameter(
            pll_fin,
            pll_config3,
            1,
            0.5)
        platformParameter = PlatformParameter(
            (designParameter.KD - 1) / (2 * designParameter.KD), # alpha
            designParameter.delta / 2, # phi_0
            L0,
            1e6
        )
        excessPhase = ExcessPhaseProcess(
            designParameter,
            platformParameter
        )
        m = EntropyModel(
            designParameter,
            platformParameter,
            size=1e6
        )
        arg_m = m.get_contributors_arg_m()
        dimension = arg_m.size
        corr_martix = m.get_contributors_corr_matrix()
        t = m.get_time_vec()
        data = np.array([(t[arg_m[i]]-t[arg_m[j]], corr_martix[i][j]) for i in range(dimension) for j in range(dimension)])
        data_sort = sorted(data, key=lambda x: x[0])
        return data_sort

    # Uncomment the following lines to generate data
    # data_sort1 = get_data(1e-10) # -100dBc
    # data_sort2 = get_data(10e-10) # -90dBc
    # x1, y1 = zip(*data_sort1)
    # x2, y2 = zip(*data_sort2)
    # np.savetxt('./data/correlation_L0/x1.txt', x1)
    # np.savetxt('./data/correlation_L0/x2.txt', x2)
    # np.savetxt('./data/correlation_L0/y1.txt', y1)
    # np.savetxt('./data/correlation_L0/y2.txt', y2)

    # Load the precomputed
    x1 = np.loadtxt('./data/correlation_L0/x1.txt')
    x2 = np.loadtxt('./data/correlation_L0/x2.txt')
    y1 = np.loadtxt('./data/correlation_L0/y1.txt')
    y2 = np.loadtxt('./data/correlation_L0/y2.txt')

    x3 = np.linspace(-1.1 * max(x1), 1.1 * max(x1),1000)
    y3 = np.exp(-2 * pi * 1e6 * np.abs(x3))

    fig, ax = plt.subplots(figsize=(12, 4))
    ax.plot(x3, y3, color=(0.5, 0.7, 0.5), label='excess phase', lw=3)
    ax.plot(x3, -y3, color=(0.5, 0.7, 0.5), lw=3)

    ax.plot(x2, y2, 'C0o:', label='bit, L0=-90dBc/Hz')
    ax.plot(x1, y1, 'C1^:', label='bit, L0=-100dBc/Hz')
    ax.fill_between(x3, y3, -y3, color=(0.8, 0.95, 0.8), alpha=0.5)
    ax.legend(loc='upper left')
    ax.set_xlabel('lag[s]')
    ax.set_ylabel('ACF')
    ax.set_ylim(-0.5,1)

    axins = inset_axes(ax, width="40%", height="60%", loc="upper right")
    axins.plot(x3, y3, color=(0.5, 0.7, 0.5), label='excess phase', lw=3)
    axins.plot(x3, -y3, color=(0.5, 0.7, 0.5), label='excess phase', lw=3)
    axins.plot(x2, y2, 'C0o:', label='bit, L0=-100dBc')
    axins.plot(x1, y1, 'C1^:', label='bit, L0=-90dBc')
    axins.set_xlim(0.35e-6, 1.2e-6)
    axins.set_ylim(-0.1, 0.2)
    axins.fill_between(x3, y3, -y3, color=(0.8, 0.95, 0.8), alpha=0.5)

    from matplotlib.ticker import MultipleLocator
    axins.xaxis.set_major_locator(MultipleLocator(0.1))
    axins.yaxis.set_major_locator(MultipleLocator(0.2))
    axins.set_xticks([])
    axins.set_yticks([])

    from mpl_toolkits.axes_grid1.inset_locator import mark_inset
    mark_inset(ax, axins, loc1=2, loc2=4, fc="none", ec="0.5")

    plt.show()

plt_correlation_L0()

- Analyze the impact of PLL bandwidth on entropy and reproduce the Fig. 9 in the paper:

In [ ]:
def plt_entropy_BW():
    data = np.genfromtxt('./data/entropy_BW/entropy_BW.txt', skip_header=2)
    BW = data[:,0] / 1e6
    H1 = data[:,1]
    Hmin = data[:,2]
    
    fig, ax = plt.subplots()
    ax.plot(BW, H1, 'C0o:', label=r'$H_1$')
    ax.plot(BW, Hmin, 'C1^:', label=r'$H_{\infty}$')
    ax.legend()
    ax.set_xlabel('Bandwidth[MHz]')
    ax.set_ylabel('Entropy')
    plt.show()

plt_entropy_BW()

- Show the correlation matrix of the sampling points for different outputs and reproduce the Fig. 10 in the paper:

In [ ]:
def plt_correlation_matrix():
    def get_data(nr_output):
        designParameter = DesignParameter(
            pll_fin,
            pll_config2,
            nr_output,
            0.5 / nr_output)
        platformParameter = PlatformParameter(
            (designParameter.KD - 1) / (2 * designParameter.KD), # alpha
            designParameter.delta / 2, # phi_0
            1e-10,
            1e6
        )
        excessPhase = ExcessPhaseProcess(
            designParameter,
            platformParameter
        )
        m = EntropyModel(
            designParameter,
            platformParameter,
            size=1e6
        )
        return m

    # Uncomment the following lines to generate data and save the correlation matrix
    # for pll_nr_output in [1, 2]:
    #     m = get_data(pll_nr_output)
    #     matrix = m.get_contributors_corr_matrix()
    #     np.savetxt('./data/correlation_matrix/correlation_matrix'+str(pll_nr_output)+'.txt', matrix)
    
    # Load the precomputed correlation matrix from file (Config2)
    for pll_nr_output in [1, 2]:
        matrix = np.loadtxt('./data/correlation_matrix/correlation_matrix'+str(pll_nr_output)+'.txt')
        cmap = 'Blues'
        plt.figure(figsize=(10, 10))
        sns.heatmap(matrix, 
                    square=True, 
                    annot=True, 
                    cmap=cmap, 
                    fmt=".2f", 
                    annot_kws={"size": 16},
                    linewidths=0.5)
        plt.xticks(fontsize=14)
        plt.yticks(fontsize=14)

        plt.show()

plt_correlation_matrix()

- Analyze the impact of the number of PLL outputs on the correlation of the sampling points and reproduce the Fig. 11 in the paper:

In [ ]:
def plt_correlation_output():
    def get_data(nr_output):
        # Config2
        designParameter = DesignParameter(
            pll_fin,
            pll_config2,
            nr_output, # {1, 2}
            0.5 / nr_output)
        platformParameter = PlatformParameter(
            (designParameter.KD - 1) / (2 * designParameter.KD), # alpha
            designParameter.delta / 2, # phi_0
            1e-10,
            1e6
        )
        excessPhase = ExcessPhaseProcess(
            designParameter,
            platformParameter
        )
        m = EntropyModel(
            designParameter,
            platformParameter,
            size=1e6
        )
        return m

    # Uncomment the following lines to generate data
    # for pll_nr_output in [1, 2]:
    #     m = get_data(pll_nr_output)
    #     shannon_entropy = m.get_shannon_entropy()
    #     min_entropy = m.get_min_entropy()
    #     arg_m = m.get_contributors_arg_m()
    #     dimension = arg_m.size
    #     corr_martix = m.get_contributors_corr_matrix()
    #     t = m.get_time_vec()
    #     data = np.array([(t[arg_m[i]]-t[arg_m[j]], corr_martix[i][j]) for i in range(dimension) for j in range(dimension)])
    #     data_sort = sorted(data, key=lambda x: x[0])
    #     np.savetxt(f'./data/correlation_output/data_sort_{pll_nr_output}.txt', data_sort)
    #     np.savetxt(f'./data/correlation_output/shannon_entropy_{pll_nr_output}.txt', [shannon_entropy])
    #     np.savetxt(f'./data/correlation_output/min_entropy_{pll_nr_output}.txt', [min_entropy])

    # Load the precomputed data
    for pll_nr_output in [1, 2]:
        data_sort = np.loadtxt(f'./data/correlation_output/data_sort_{pll_nr_output}.txt')
        shannon_entropy = np.loadtxt(f'./data/correlation_output/shannon_entropy_{pll_nr_output}.txt')
        min_entropy = np.loadtxt(f'./data/correlation_output/min_entropy_{pll_nr_output}.txt')

        fig, ax = plt.subplots()
        x1, y1 = zip(*data_sort)
        ax.plot(x1, y1, 'C0o:', label='bit')
        
        x2 = np.linspace(-1.1 * max(x1), 1.1 * max(x1),1000)
        y2 = np.exp(-2 * pi * 1e6 * np.abs(x2))
        ax.plot(x2, y2, 'C2--', label='excess phase')

        ax.text(4e-7, 0.6, f'$H_1 = {shannon_entropy:.4f}$')
        ax.text(4e-7, 0.4, f'$H_{{\\infty}} = {min_entropy:.4f}$')
        ax.legend()
        ax.set_xlabel('lag[s]')
        ax.set_ylabel('ACF')
        plt.show()

plt_correlation_output()

### Case study

Build simulation model and show basic parameters for `pll_confg1` with the default parameters.

In [ ]:
# Initialize model parameters
designParameter = DesignParameter(
    pll_fin,
    pll_config1,
    pll_nr_output,
    pll_phase_latency)

platformParameter = PlatformParameter(
    (designParameter.KD - 1) / (2 * designParameter.KD),
    designParameter.delta / 2,
    pll_L0,
    pll_BW
)

m = EntropyModel(
    designParameter,
    platformParameter,
    size=1e4 # for quick test, use 1e6 for real test
)

# Display parameters
print("Design Parameter:")
display_class_members(designParameter)
print("\nPlatform Parameter:")
display_class_members(platformParameter)
print("\nContributors args:")
m.print_contributors_args()
print("\nEntropy Study:")
m.print_entropy()

# Generate plots
print("Plotting correlation matrix...")
m.plt_contributors_corr_matrix()

print("Plotting lag correlation...")
m.plt_lag_corr()